Các bạn sử dụng tập dữ liệu của bài tập 5, mô hình dữ liệu x theo dạng bậc 6 và làm các công việc sau:

1. Chia dữ liệu ra thành 2 tập: training (70%) và validation (30%). Phải đảm bảo việc chia dữ liệu là ngẫu nhiên và tỷ lệ positive và negative cân bằng.

2. Viết chương trình cho phép học các tham số của mô hình phân loại phi tuyến trên có sử dụng regularization L2 và L1.

3. Tính J ở mỗi vòng lặp cho cả hai tập, chọn điểm dừng phù hợp.

4. Thay đổi lambda và tính J cho mỗi lambda tương ứng cho cả hai tập. Vẽ biểu đồ quan hệ giữa J và lambda từ đó lựa chọn lambda phù hợp.

In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [2]:
ex5_data = pd.read_csv('ex5data.txt', header= None).to_numpy()

In [3]:
ex5_data.shape

(118, 3)

In [4]:
x1, x2, y = ex5_data[:, 0], ex5_data[:, 1], ex5_data[:, 2]  

In [5]:
colors = np.where(y == 1, 'red', 'blue')

In [6]:
fig = go.Figure()

# Plot data points
fig.add_trace(go.Scatter(
    x=x1, 
    y=x2, 
    mode='markers', 
    marker=dict(color=colors, size=10),
    name='Data Points'
))

fig.update_xaxes(title= 'x1')
fig.update_yaxes(title= 'x2')


In [7]:
x = ex5_data[:, :2]

In [9]:
x_pos = x[y == 1]
x_neg = x[y == 0]

In [10]:
rng = np.random.default_rng()

rng.shuffle(x_pos)
rng.shuffle(x_neg)

In [11]:
len_train = int(0.7 * len(y))
len_valid = len(y) - len_train

In [13]:
x_train = np.vstack((x_pos[0: int(len_train / 2)], x_neg[0: len_train - int(len_train / 2)]))
x_valid = np.vstack((x_pos[0: int(len_train / 2)], x_neg[(len_train - int(len_train / 2))]))

In [15]:
np.random.shuffle(x_train)
np.random.shuffle(x_valid)

In [23]:
n = 0
X_mapFeature = np.ones((118, 28))

for order in range(1, 7, 1):
    for i in range(0, order+1):

        x12 = x1**(order - i) * x2**(i)
        X_mapFeature[:, n] = x12
        n = n + 1

        # print(f'x1:^{(order - i)} - x2^{(i)}')
        # print(n)

In [ ]:
class Logistic_Regression_Multivariables:
    def __init__(self, *, number_of_feature: int) -> None:
        self.number_of_features = number_of_feature

    def normalize_vector(self, vector: np.ndarray) -> np.ndarray:
        mean = np.mean(vector)
        std = np.std(vector)
        if std == 0:
            return vector - mean
        return (vector - mean) / std
    
    def normalize_input(self, *, X: np.ndarray) -> tuple:
        norm_X = np.apply_along_axis(self.normalize_vector, arr=X, axis=0).reshape(-1, self.number_of_features)
        return norm_X

    def add_ones_columns(self, *, normalized_input: np.ndarray) -> np.ndarray:
        ones = np.ones(len(normalized_input)).reshape(-1, 1)
        x_add = np.hstack((ones, normalized_input))
        return x_add

    def predict(self, *, theta: np.ndarray, normalized_input: np.ndarray) -> np.ndarray:
        y_pred = np.matmul(normalized_input, theta)
        y_pred = 1/(1 + np.exp(-y_pred))
        return y_pred
    
    def compute_loss(self, *, y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
        m = len(y_true)
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        J = np.sum(- y_true*np.log(y_pred) - (1 - y_true)*np.log(1 - y_pred)) / (m)
        return J
    
    def update_params(self, *, theta: np.ndarray, lr: float, y_pred: np.ndarray, 
                      y_true: np.ndarray, normalized_input: np.ndarray) -> np.ndarray:
        m = len(y_true)
        E = y_pred - y_true
        dJ_dtheta = np.dot(normalized_input.T, E) / (m)
        theta_updated = theta - lr*dJ_dtheta
        return theta_updated
    
    def train(self, *, epochs: int, theta: np.ndarray, input: np.ndarray, 
              output: np.ndarray, lr: float, plot_graph: False, color: list, time_delay: float) -> np.ndarray:
        output = output.reshape(-1, 1)
        normalized_input = self.normalize_input(X= input)
        normalized_input_with_ones = self.add_ones_columns(normalized_input= normalized_input)

        J_array = np.array([])
        
        for epoch in range(epochs):
            y_pred = self.predict(theta= theta, normalized_input= normalized_input_with_ones)

            J = self.compute_loss(y_true= output, y_pred= y_pred)
            J_array = np.append(arr= J_array, values= J)

            theta = self.update_params(theta= theta, lr= lr, y_pred= y_pred, 
                                       y_true= output, normalized_input= normalized_input_with_ones)

            
    
        return J_array, theta

In [ ]:
num_feature = X_mapFeature.shape[1]
Logistic_Regression = Logistic_Regression_Multivariables(number_of_feature= num_feature)

normalized_input = Logistic_Regression.normalize_input(X= X_mapFeature)
normalized_input_with_ones = Logistic_Regression.add_ones_columns(normalized_input= normalized_input)


In [ ]:
np.random.seed(1)

theta_init = np.random.rand(num_feature + 1, 1)
theta_init